In [ ]:
# Packages
import numpy as np
import time
from queue import Queue

# Helper Variables
allfeatures = [1,2,3,4] # only integers at the moment

class node:
  def __init__(self, _features:set = set(), score:float = 0.0, classification:float = 0.0):
    self.classification = classification # float
    self.features_set = _features # set
    self.features = [] # list???
    self.parents = [] # node
    self.children = [] # nodes
    self.score = 0.0 # float, ig

  def __str__(self):
    return str(self.features_set)

  def formattedStr(self):
    return f"Classification: {int(self.classification)} | Features: {str(self.features)}"

# this is the filler snippet, dont use it
def evaluate(n : node):
  n.score = round(np.random.rand() * 100, 1)
  return n.score

def eval(n:node, instances:list, validator:Validator, selected_features:list):

  features = []

  for item in n.features_set:
    features.append("Feature " + str(item))

  # Get default rate
  if(len(features) <= 0):
    temp_dict = {}
    for instance in instances:
      if(instance.classification in temp_dict):
        temp_dict[instance.classification] += 1
      else:
        temp_dict[instance.classification] = 1

    most_Common_Class_Size = 0

    for classification in temp_dict:
      if temp_dict[classification] > most_Common_Class_Size:
        most_Common_Class_Size = temp_dict[classification]
    default_rate = most_Common_Class_Size / len(instances)

    n.score = default_rate
  # Use k-folds 1-NN for score.
  else:
    n.score = validator.validate(selected_features)
  return n.score

class tree:
  def __init__(self, validator:Validator, selected_features:list, root:node = node(), tail:node = node()):
    self.root = root
    self.tail = tail
    self.validator = validator
    self.features = selected_features

  def __str__(self):
    returnstr = ""
    queue1 = Queue()
    queue2 = Queue()
    queue1.put(self.root)
    while(not queue1.empty()):
      # handle dupes and add children to next batch
      nodesToPrint = set()

      # Print all nodes on current depth
      while(not queue1.empty()):
        curr = queue1.get()
        for child in curr.children:
          if tuple(child.features_set) not in nodesToPrint:
            queue2.put(child)
          nodesToPrint.add(tuple(child.features_set))
        returnstr += str(curr.features_set) + " "

      # prepare next batch of children
      while(not queue2.empty()):
        queue1.put(queue2.get())
      returnstr += "\n"
    return returnstr

  def fill(self, instances:list = list()):
    queue1 = Queue()
    queue2 = Queue()
    queue1.put(self.root)

    # Use 2 queues for bootleg breadth first search, recursion is a sign of weakness
    while(not queue1.empty()):
      currFeatureSet = set()
      while(not queue1.empty()):
        curr = queue1.get()

        if instances == 0 :
          evaluate(curr)
        else:
          eval(curr, instances, self.validator, self.features)

        # If we have a node in the list that is all our entire feature set we stop
        if(set(self.features).difference(curr.features_set) == set()):
          self.tail = curr
          return

        # Use 2nd queue to iterate through parents in future
        queue2.put(curr)

        # Gather all possible features generated from parent nodes
        for feature in self.features:
          newfeatures = set()
          for currentfeature in curr.features_set:
            newfeatures = curr.features_set.copy()
            newfeatures.add(currentfeature)
          newfeatures.add(feature)

          # Create a set of unique tuples
          if(newfeatures > curr.features_set):
            # print(f"{newfeatures}: added {feature}")
            if(len(newfeatures) > 1):
              currFeatureSet.add(tuple(newfeatures))
            else:
              currFeatureSet.update(newfeatures)

      # create child nodes with all new feature possibilities given parent nodes
      children = []
      for feature in currFeatureSet:
        if(type(feature) == int):
          child = node({feature})
        else:
          child = node(set(feature))
        children.append(child)
        queue1.put(child)

      ## if the child is a super set of the parent, assign child to that parent
      while(not queue2.empty()):
        curr = queue2.get()
        # print(f"\nSetting child for {curr}")
        for child in children:
          # if the parent node is a subset of a child node
          if(curr.features_set < child.features_set):
            child.parents.append(curr)
            curr.children.append(child)
  #------------------------------------ END FUNC

  def forwardSearch(self, instances):
    bestState = self.root         # will be returned at the end, hopefully isnt self.root
    bestStateScore = 0

    if instances == 0:
      bestStateScore = evaluate(bestState)
    else:
      bestStateScore = eval(bestState, instances, self.validator, self.features)

    children = bestState.children # start by seeding the 1-feature combination nodes

    print("Running neareset neighbor with no features (default rate), using \"leaving-one-out\" evaluation, I get an accuracy of " + str(bestStateScore) + "%\n")

    print("Beginning search\n")
    for i in range(len(self.features)):  # we're only gonna be searching so many times cuz it's greedy
      bestScoreAtDepth = -1
      bestStateAtDepth = node()

      for child in children:
        score = 0
        if instances == 0:
          score = evaluate(child)
        else:
          score = eval(child, instances, self.validator, self.features)

        print("\tUsing feature(s) ", end = "")
        print(child, end = "")
        print(" accuracy is " + str(score) + "%")

        if score > bestScoreAtDepth:
          bestScoreAtDepth = score
          bestStateAtDepth = child

      # -----------------------
      print()

      if bestScoreAtDepth >= bestStateScore: # better than previous bestState
        bestState = bestStateAtDepth
        bestStateScore = bestScoreAtDepth
        print("\nFeature set ", end = "")
        print(bestStateAtDepth, end = "")
        print(" was best, accuracy is " + str(bestScoreAtDepth) + "%\n")
      else:
        print("(Warning, Accuracy has decreased! Continuing search in case of local maxima)")

      children = bestStateAtDepth.children # update for next iteration
    # --------------------------------------

    print("Finished search, best feature subset is ", end = "")
    print(bestState, end = "")
    print(", which has an accuracy of " + str(bestStateScore) + "%")
    return bestState
  # ---------------------------------------------- END FUNC

  def backwardSearch(self, instances):
    bestState = self.tail
    bestStateScore = 0
    if instances == 0:
      bestStateScore = evaluate(bestState)
    else:
      bestStateScore = eval(bestState, instances, self.validator, self.features)

    parents = bestState.parents

    print("All features, 'random' evaluation: " + str(bestStateScore) + "%\n")
    print("Start search\n")
    for i in range(len(self.features)):
      # if best state is the tail node
      bestScoreAtDepth = -1
      bestStateAtDepth = node()

      for parent in parents:
        score = 0
        if instances == 0:
          score = evaluate(parent)
        else:
          score = eval(parent, instances, self.validator, self.features)

        print("\tUsing feature(s) ", end = "")
        print(parent, end = "")
        print(" accuracy is " + str(score) + "%")

        if score > bestScoreAtDepth:
          bestScoreAtDepth = score
          bestStateAtDepth = parent
      # -----------------------
      print()

      if bestScoreAtDepth >= bestStateScore: # better than previous bestState
        bestState = bestStateAtDepth
        bestStateScore = bestScoreAtDepth
        print("Feature set ", end = "")
        print(bestState, end = "")
        print(" was best, accuracy is " + str(bestStateScore) + "%\n")
      else:
        print("Warning, Accuracy has decreased!\n")

      parents = bestStateAtDepth.parents # update for next iteration
    # --------------------------------------

    print("Finished search, best feature subset is ", end = "")
    print(bestState, end = "")
    print(", which has an accuracy of " + str(bestStateScore) + "%")
    return bestState
  # ---------------------------------------------- END FUNC

  # explore all good nodes that are within epsilon range of the best node.
  # Epsilon is a range(in percentage) of the best feature set to determine
  # if a feature set is within worth exploring.

  # epsilon = 0, forward greedy search
  # epsilon = 100, Breadth First Search
  def SpecialYumYum(self, epsilon:float = 5):
    queue1 = Queue()
    queue2 = Queue()
    bestNode = self.root
    queue1.put(bestNode)
    visitedNodes = set()
    print(f"No features, 'random' evaluation: {bestNode.score}%\n")
    print("Start search\n")
    # Use 2 queues for bootleg breadth first search, recursion is a sign of weakness
    while(not queue1.empty()):
      bestNodeAtDepth = node()
      while(not queue1.empty()):
        newBestNodes = []
        bestNodes = []
        validNodes = []
        curr = queue1.get()

        # Conditional statements for loop
        if len(curr.children) == 0:
          break
        if tuple(curr.features_set) in visitedNodes:
          continue
        visitedNodes.add(tuple(curr.features_set))

        # If find child with best score at current depth, compare to the best node we have
        print(f"For {curr}, Score: {curr.score}%\n")
        for child in curr.children:
          if tuple(child.features_set) not in visitedNodes:
            print(f"\tUsing feature(s) {child} accuracy is {child.score}%")
            if child.score > bestNode.score - epsilon:
              validNodes.append(child)
            if child.score > bestNodeAtDepth.score:
              bestNodeAtDepth = child
        print()

        for child in validNodes:
          # Create new best if current node is better than all nodes by more than epsilon
          if child.score > bestNode.score:
            print(f"Feature set {child} is better than best node({bestNode}), new best accuracy is {child.score}%")
            bestNode = child
            newBestNodes = [child]

            # Empty out next best candidates in range epsilon of new best
            while not queue2.empty():
              queue2.get()

            # Create the set
            for n in bestNodes:
              if n.score > bestNode.score - epsilon:
                newBestNodes.append(n)
            bestNodes = newBestNodes.copy()

          # Add node to list of bestNodes at this depth
          elif child.score > bestNode.score - epsilon:
            print(f"Feature set {child} is within epsilon of best node({bestNode}), accuracy is {child.score}%")
            bestNodes.append(child)

          # Node not worth exploring
          else:
            print(f"Feature set {child} is not within epsilon of best node({bestNode}), accuracy is {child.score}%")

        # Queue up next best nodes at the next depth, or if no node was worthwhile we just pick the best node at that depth
        print(f"\nFeature set {bestNodeAtDepth} was the best node available, accuracy is {bestNodeAtDepth.score}%")
        if len(bestNodes) > 0:
          for bnode in bestNodes:
            queue2.put(bnode)
        else:
          queue2.put(bestNodeAtDepth)
        print()

      while(not queue2.empty()):
        queue1.put(queue2.get())
    print(f"Finished search, best feature subset is {bestNode}, which has an accuracy of {bestNode.score}%")
    return bestNode
  # ---------------------------------------------- END FUNC

#-------------------------------------------------------------- END CLASS

In [ ]:
from re import A
import math
import pandas as pd

# Classifying Time
class Classifier:
  def __init__(self):
    self.training_data = list()
    self.test_data = list()
    self.frame = pd.DataFrame()

  def __str__(self):
    returnstr = "{:<145}{:<145}\n".format("Training", "Test")
    for i in range(max(len(self.training_data), len(self.test_data))):
      if(i < len(self.training_data)):
        traindata = self.training_data[i].formattedStr()
      else:
        traindata = ''

      if(i < len(self.test_data)):
        testdata = self.test_data[i].formattedStr()
      else:
        testdata = ''
      returnstr += "{:<145}{:<145}\n".format(traindata, testdata)
    return returnstr

  # Can input test with split(0 to 1) %, split = .25 means 25% of data is test data,
  # 75% becomes training data. The test array will always be assigned starting from
  # index 0 to the desired test data amount.

  # train() will append all data to the existing training set
  def train(self, data:list=[], split:float=0, testAmount:int=0, dataframe = None):
    # If we decide to use a panda then assign it for future use, hope that user has understanding
    if isinstance(dataframe, pd.DataFrame):
      self.frame = dataframe
      if(len(data) == 0):
        raise Exception("Recieved DataFrame but no data. Set data = list(node)")

    if(testAmount > 0):
      size = testAmount
    else:
      size = int(len(data) * split)

    for i in range(len(data)):
      if(i < size):
        self.test_data.append(data[i])
      else:
        self.training_data.append(data[i])

  # reset the model
  def reset(self):
    self.training_data = []
    self.test_data = []

  # Euclidean distance between two instances
  def Euclidean_Distance_For_Numbers(self, training_node:node, test_node:node):
    sum = 0
    for feature in range(len(training_node.features)):
      difference = training_node.features[feature] - test_node.features[feature]
      sum += difference * difference
    return math.sqrt(sum)

  # Euclidean distance using pandas dataframe and a list. Columns of list is same as columns of dataframe
  def Euclidean_Distance_For_Pandas(self, test:pd.DataFrame, frame:pd.DataFrame):
    # strip Class Information for euclidean
    features = []
    for feature in frame:
      if feature != "Class Label":
        features.append(feature)

    a = test[features].values.squeeze()
    b = frame[features]
    difference = b - a
    square = difference * difference
    sum = np.sum(square,axis=1)
    return np.sqrt(sum)

  # The test function can take in a list of test data and return the closest instance
  # to each given test instance
  def test(self, testdata = None):
    start_time = time.time()
    returnClassification = []
    formatstr = "{:<15}{:<15}"
    # print(formatstr.format("Test Instance", "Classifying Instance"))

    # Test against the test set that was last used/currently stored
    if testdata is None:
      for test_instance in self.test_data:
        nearestNeighbor = math.inf
        bestClassifier = node()
        for classifying_instance in self.training_data:
          distance = self.Euclidean_Distance_For_Numbers(classifying_instance, test_instance)
          if distance < nearestNeighbor:
            bestClassifier = classifying_instance
            nearestNeighbor = distance
        print(formatstr.format(test_instance.classification, bestClassifier.classification))
        returnClassification.append(bestClassifier)

    # If a list is given test each instance in the list
    elif isinstance(testdata, list):
      self.test_data = testdata.copy()
      for test_instance in self.test_data:
        nearestNeighbor = math.inf
        bestClassifier = node()
        for classifying_instance in self.training_data:
          distance = self.Euclidean_Distance_For_Numbers(classifying_instance, test_instance)
          if distance < nearestNeighbor:
            bestClassifier = classifying_instance
            nearestNeighbor = distance
        print(formatstr.format(test_instance.classification, bestClassifier.classification))
        returnClassification.append(bestClassifier)

    # Use Pandas dataframe
    elif isinstance(testdata, pd.DataFrame):
      distances = self.Euclidean_Distance_For_Pandas(testdata, self.frame)
      # print(formatstr.format(testdata["Class Label"].to_list()[0], int(self.training_data[distances.argmin()].classification)))
      returnClassification.append(self.training_data[distances.idxmin()])

    # If given a single node
    else:
      nearestNeighbor = math.inf
      bestClassifier = node()
      for classifying_instance in self.training_data:
        distance = self.Euclidean_Distance_For_Numbers(classifying_instance, testdata)
        if distance < nearestNeighbor:
          bestClassifier = classifying_instance
          nearestNeighbor = distance
      print(formatstr.format(testdata.classification, bestClassifier.classification))
      returnClassification.append(bestClassifier)
    end_time = time.time()
    # print(f"Time elapsed: {end_time - start_time}\n")
    return returnClassification

In [ ]:
import time
import pandas as pd
from sklearn.preprocessing import StandardScaler


class Validator:
  def __init__(self, classifier:Classifier, features:set, instances:list[node]):
    self.classifier = classifier
    self.features = list(features)
    self.instances = instances
    self.data = []

    for instance in instances:
      self.data.append(instance.features)
    self.data = np.array(self.data)

    self.frame = pd.DataFrame({"Class Label": [int(i.classification) for i in instances]})
    for feature in range(len(instances[0].features)):
      feature_frame = pd.DataFrame({"Feature " + str(feature + 1): self.data[:,feature]})
      self.frame = pd.concat([self.frame, feature_frame], axis = 1)

    print("Please wait while I normalize the data...  ", end="")
    scaler = StandardScaler()
    columns_to_standardize = self.frame.columns.difference(['Class Label'])
    self.frame[columns_to_standardize] = (self.frame[columns_to_standardize] - self.frame[columns_to_standardize].mean()) / self.frame[columns_to_standardize].std()
    print("Done!\n")
  # -------------------

  def __str__(self):
    return str(self.frame)
  # -------------------

  # Given a set of features, k-folds cross validation on feature set. Refer to input code further below for usage.
  def validate(self, selected_features:list):
    validation_table = []

    features = []
    for item in selected_features:
      features.append("Feature " + str(item))
    # train model with features
    selected_frame = self.frame[["Class Label"] + features]

    for i in range(len(self.instances)):
      # target specific instance
      test_instance_mask = selected_frame.index.isin([i])

      # Get instance we for test set, I use a panda frame since its easier to do large amounts of data
      test_instance = selected_frame[test_instance_mask]
      frame_masked = selected_frame[~test_instance_mask]

      # train and test
      self.classifier.train(data = self.instances, dataframe = frame_masked)
      classifierList = self.classifier.test(test_instance)

      # technically just 1 instance, but still use loop to validate. Add validation to table.
      for classified_instance in classifierList:
        validation_table.append(classified_instance.classification == self.instances[i].classification)

    # Now we have a filled valdation table of true and false(correct or incorrect classification)
    return sum(validation_table) / len(validation_table)

In [ ]:
# Reading Text Time
small_file = list()
large_file = list()
small = r"small-test-dataset.txt"
large = r"large-test-dataset.txt"
with open(small, "r") as f:
  for line in f:
    small_file.append(line.split())
  f.close()

with open(large, "r") as f:
  for line in f:
    large_file.append(line.split())
  f.close()

# Conversion Time part 1
small_contents = []
for instance in small_file:
  small_contents.append(list(map(float, instance)))

large_contents = []
for instance in large_file:
  large_contents.append(list(map(float, instance)))

# Conversion Time Part 2
## ------------------------------------------------------------------------------------------------------------------------------------
## MASSIVE NOTE: if you notice something is wrong with the data re run the data collection starting from when we read in the file. It is just a collab thing.
##                - Classification should be either a 1 or 2 for example
## ------------------------------------------------------------------------------------------------------------------------------------
small_instances = []
for instance in small_contents:
  n = node()
  n.classification = instance.pop(0)
  n.features = instance.copy()
  n.features_set = set(n.features)
  small_instances.append(n)

large_instances = []
for instance in large_contents:
  n = node()
  n.classification = instance.pop(0)
  n.features = instance.copy()
  n.features_set = set(n.features)
  large_instances.append(n)

smallClassifier = Classifier()

print(f"Small Dataset: small-test-dataset.txt (Has {len(small_instances)} instances and {len(small_instances[0].features)} features)")
userInput = input(f"Enter the desired features (1 - {len(small_instances[0].features)}), separated by spaces: ")
print("\r")
userInputAsList = userInput.split()

# Use set since we dont want to look at duplicate features.
features = set()
for item in userInputAsList:
  features.add(item)

smallValidator = Validator(smallClassifier, features, small_instances)

print("Data Frame for small data set:\n")
print(smallValidator)

small_accuracy = smallValidator.validate(selected_features = list(features))
print(f"\t- If you use only {[int(feature) for feature in userInputAsList]}, accuracy should be about {small_accuracy}")

largeClassifier = Classifier()

print("Data Frame for large data set:\n")

print(f"Large Dataset: large-test-dataset.txt (Has {len(large_instances)} instances and {len(large_instances[0].features)} features)")
userInput = input(f"Enter the desired features (1 - {len(large_instances[0].features)}), separated by spaces: ")
print("\r")
userInputAsList = userInput.split()

# Use set since we dont want to look at duplicate features.
features = set()
for item in userInputAsList:
  features.add(item)

largeValidator = Validator(largeClassifier, features, large_instances)
print(largeValidator)
large_accuracy = largeValidator.validate(selected_features = list(features))
print(f"\t- If you use only {[int(feature) for feature in userInputAsList]}, accuracy should be about {large_accuracy}")

In [ ]:
# read in data --------------------------------------------
small_file = list()
large_file = list()
small = r"CS170_Spring_2024_Small_data__29.txt"
large = r"CS170_Spring_2024_Large_data__29.txt"

with open(small, "r") as f:
  for line in f:
    small_file.append(line.split())
  f.close()

with open(large, "r") as f:
  for line in f:
    large_file.append(line.split())
  f.close()

# convert classification -----------------------------------
small_contents = []
for instance in small_file:
  small_contents.append(list(map(float, instance)))

large_contents = []
for instance in large_file:
  large_contents.append(list(map(float, instance)))
print()

# convert features -----------------------------------------
small_instances = []
for instance in small_contents:
  n = node()
  n.classification = instance.pop(0)
  n.features = instance.copy()
  n.features_set = set(n.features)
  small_instances.append(n)

large_instances = []
for instance in large_contents:
  n = node()
  n.classification = instance.pop(0)
  n.features = instance.copy()
  n.features_set = set(n.features)
  large_instances.append(n)

# actually test ----------------------------------------------------------------
dataset = input("Test small or large dataset? (1 or 2): ")
algo = input("Which algo?\n1 - Forward Search\n2 - Backward Search\n3 - Justin's Special\n")

theInstances = []
features = []

if dataset == "1":
  # get desired features
  print(f"Small Dataset (has {len(small_instances)} instances and {len(small_instances[0].features)} features)")
  features = input(f"Enter the desired features (1 - {len(small_instances[0].features)}), separated by spaces. Enter nothing to use all features: ")
  # prep tree construction
  numFeatures = len(small_instances[0].features)
  for i in range(int(numFeatures)):
    allfeatures.append(i + 1)

  theInstances = small_instances
elif dataset == "2":
  # get desired features
  print(f"Large Dataset (has {len(large_instances)} instances and {len(large_instances[0].features)} features)")
  features = input(f"Enter the desired features (1 - {len(small_instances[0].features)}), separated by spaces. Enter nothing to use all features: ")
  # prep tree construction
  numFeatures = len(large_instances[0].features)
  for i in range(int(numFeatures)):
    allfeatures.append(i + 1)

  theInstances = large_instances
else:
  print("Unrecognized input for dataset!")
  quit()
classifier = Classifier()
validator = Validator(classifier, set(allfeatures), theInstances)

selected_features = []
t1 = None
if len(features) > 0:
  t1 = tree(validator, features)
else:
  t1 = tree(validator, allfeatures)
# construct the tree
t1.fill(theInstances)
stateFound = node()

if algo == "1":
  stateFound = t1.forwardSearch(theInstances)
elif algo == "2":
  stateFound = t1.backwardSearch(theInstances)
elif algo == "3":
  print("Justin's Special requires an epsilon that is used to explore nodes that are within range of the best state explored. 0 means Forward Selection, 100 means search entire tree")
  epsilon = input("Provide epsilon(Percentage range(ex. 5, 10, 20, 50)): ")
  print()
  stateFound = t1.SpecialYumYum(float(epsilon))
else:
  print("Unrecognized input for algorithm!")
  quit()


Test small or large dataset? (1 or 2): 1
Which algo?
1 - Forward Search
2 - Backward Search
3 - Justin's Special
1
Small Dataset (has 100 instances and 10 features)
Enter the desired features (1 - 10), separated by spaces. Enter nothing to use all features: 
Please wait while I normalize the data...  Done!



KeyboardInterrupt: 